In [50]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import time
start = time.time()

In [52]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Read the CSV file
df = pd.read_csv('grib_data.csv', sep=',')

# Convert datetime columns to numeric (e.g., Unix timestamp)
def convert_to_numeric(df, column_name):
    df[column_name] = pd.to_datetime(df[column_name], errors='coerce').astype(np.int64) / 1e9  # Convert to Unix timestamp

# Convert relevant columns
for col in ['time', 'step', 'valid_time']:
    convert_to_numeric(df, col)

# Convert all columns to float
df = df.astype(float)

# Split the dataset into training and testing sets
train, test = train_test_split(df, test_size=0.30, random_state=2)

# Sample the datasets
train_set = train.sample(160, random_state=42)
test_set = test.sample(40, random_state=42)

# Set random seed
np.random.seed(42)


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359775 entries, 0 to 359774
Data columns (total 70 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   time                 359775 non-null  float64
 1   latitude             359775 non-null  float64
 2   longitude            359775 non-null  float64
 3   number               359775 non-null  float64
 4   step                 359775 non-null  float64
 5   surface              359775 non-null  float64
 6   valid_time           359775 non-null  float64
 7   u100                 359775 non-null  float64
 8   v100                 359775 non-null  float64
 9   u10n                 359775 non-null  float64
 10  u10                  359775 non-null  float64
 11  v10n                 359775 non-null  float64
 12  v10                  359775 non-null  float64
 13  d2m                  359775 non-null  float64
 14  t2m                  359775 non-null  float64
 15  meanSea          

In [54]:
df.describe()

,time,latitude,longitude,number,step,surface,valid_time,u100,v100,u10n,...,stl1,slt,sdfor,sdor,sp,tsn,tcc,tciw,tclw,tco3
count,3.597750e+05,359775.000000,359775.000000,359775.0,3.597750e+05,359775.0,3.597750e+05,359775.000000,359775.000000,359775.000000,...,359775.000000,359775.000000,359775.000000,359775.000000,359775.000000,359775.000000,359775.000000,359775.000000,359775.000000,359775.000000
mean,1.705108e+09,21.990000,82.670000,0.0,-9.223372e+09,0.0,1.705108e+09,-1.389359,-1.447951,-1.265897,...,285.773462,1.889584,67.941117,94.489307,89724.177824,276.160228,0.418842,0.005182,0.026459,0.005839
std,6.230401e+05,8.876479,8.443451,0.0,1.587298e-02,0.0,6.230401e+05,2.909469,2.929684,2.141267,...,16.737633,1.581977,112.114888,147.960510,17612.564356,22.047783,0.378391,0.027872,0.060537,0.000687
min,1.704071e+09,6.740000,68.170000,0.0,-9.223372e+09,0.0,1.704071e+09,-13.360776,-12.394982,-11.429073,...,241.089450,0.000000,0.000000,0.000000,48842.492000,223.589070,0.000000,0.000000,0.000000,0.004930
25%,1.704589e+09,14.240000,75.420000,0.0,-9.223372e+09,0.0,1.704589e+09,-3.436470,-3.569078,-2.226872,...,276.682680,0.000000,0.000000,0.000000,86302.818000,272.347230,0.029724,0.000000,0.000000,0.005292
50%,1.705108e+09,21.990000,82.670000,0.0,-9.223372e+09,0.0,1.705108e+09,-1.179940,-1.237203,-0.480583,...,290.270630,2.000000,12.004761,20.667542,99471.280000,273.161070,0.322571,0.000004,0.002869,0.005592
75%,1.705626e+09,29.740000,89.920000,0.0,-9.223372e+09,0.0,1.705626e+09,0.523396,0.577265,0.125910,...,300.553815,3.000000,88.463560,130.827580,101245.766000,300.468020,0.816681,0.000853,0.027557,0.006239
max,1.706144e+09,37.240000,97.170000,0.0,-9.223372e+09,0.0,1.706144e+09,12.678922,9.792582,7.389265,...,303.333280,7.000000,692.202330,923.084700,102031.625000,303.332950,1.000000,1.421082,1.204315,0.009016


In [58]:
from sklearn.preprocessing import StandardScaler

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(train_set)
X_test = scaler.transform(test_set)


In [60]:
import pennylane as qml
from pennylane.templates import AngleEmbedding

# Define a quantum device
dev = qml.device("default.qubit", wires=4)

# Define a quantum node
@qml.qnode(dev)
def quantum_model(params, x):
    AngleEmbedding(x, wires=[0, 1, 2, 3])
    qml.templates.layers.BasicEntanglerLayers(params, wires=[0, 1, 2, 3])
    return qml.expval(qml.PauliZ(0))

# Example function to get quantum predictions
def get_predictions(X):
    predictions = []
    for x in X:
        params = np.random.rand(3, 4)  # Example: Random params, adjust as necessary
        pred = quantum_model(params, x)
        predictions.append(pred)
    return np.array(predictions)


In [78]:
import pandas as pd
import numpy as np
import pennylane as qml
from pennylane import numpy as pnp
from pennylane import QNode, Hadamard, CNOT, RX, RZ, Z
from pennylane.templates import AngleEmbedding
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import time

# Measure time to track execution
start = time.time()

# Read the CSV file
df = pd.read_csv('grib_data.csv', sep=',')

# Convert datetime columns to numeric (e.g., Unix timestamp)
def convert_to_numeric(df, column_name):
    df[column_name] = pd.to_datetime(df[column_name], errors='coerce').astype(np.int64) / 1e9  # Convert to Unix timestamp

# Convert relevant columns
for col in ['time', 'step', 'valid_time']:
    convert_to_numeric(df, col)

# Convert all columns to float
df = df.astype(float)

# Handle missing values
imputer = SimpleImputer(strategy='mean')  # You can also use other strategies like median or most_frequent
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

# Split the dataset into training and testing sets
train, test = train_test_split(df_imputed, test_size=0.30, random_state=2)

# Sample the datasets
train_set = train.sample(160, random_state=42)
test_set = test.sample(40, random_state=42)

# Set random seed
np.random.seed(42)

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(train_set.drop(columns=['latitude', 'longitude']))
X_test = scaler.transform(test_set.drop(columns=['latitude', 'longitude']))

# Extract latitude and longitude for anomalies
lat_train = train_set[['latitude', 'longitude']].values
lat_test = test_set[['latitude', 'longitude']].values

# Number of qubits
n_qubits = 3  # Increase to accommodate QPE and HHL

# Apply PCA to reduce features to match the number of qubits
pca = PCA(n_components=n_qubits - 1)  # Use n_qubits - 1 for feature reduction
X_train_reduced = pca.fit_transform(X_train)
X_test_reduced = pca.transform(X_test)

# Define matrix and vector for HHL
def get_matrix_and_vector():
    # Example matrix A (2x2) and vector b
    A = np.array([[1, 2], [3, 4]])
    b = np.array([1, 0])  # Example vector
    return A, b

# Define the QPE circuit
dev_qpe = qml.device("default.qubit", wires=3)  # Number of qubits for QPE

@qml.qnode(dev_qpe)
def qpe_circuit():
    # Apply Hadamard gates to the first two qubits
    for i in range(2):
        Hadamard(wires=i)
    
    # Apply controlled operations (simulated)
    for i in range(2):
        CNOT(wires=[i, 2])

    # Apply inverse QFT (simplified)
    for i in range(2):
        RZ(np.pi / 4, wires=i)

    return qml.expval(qml.PauliZ(0))

# Define HHL operations
dev_hhl = qml.device("default.qubit", wires=3)  # Number of qubits for HHL

@qml.qnode(dev_hhl)
def hhl_algorithm(params, b):
    # Apply AngleEmbedding to vector b
    AngleEmbedding(b, wires=[0, 1])

    # Apply HHL operations (placeholders here)
    for i in range(2):
        RX(params[i], wires=i)
    
    # Simulate controlled rotations (placeholders)
    for i in range(2):
        RZ(params[i], wires=2)

    return qml.expval(qml.PauliZ(0))

def solve_linear_system(A, b):
    # Perform QPE
    qpe_result = qpe_circuit()

    # Solve using HHL
    params = [np.pi / 4, np.pi / 2]  # Example parameters
    return hhl_algorithm(params, b)

def get_predictions(X):
    predictions = []
    for x in X:
        A, b = get_matrix_and_vector()  # Get matrix A and vector b
        pred = solve_linear_system(A, x)
        predictions.append(pred)
    return np.array(predictions)

# Generate predictions
y_train_pred = get_predictions(X_train_reduced)
y_test_pred = get_predictions(X_test_reduced)

# Define an anomaly threshold
threshold = np.percentile(y_train_pred, 95)  # Example threshold

# Identify anomalies
train_anomalies = y_train_pred > threshold
test_anomalies = y_test_pred > threshold

# Attach confidence scores
train_confidence = y_train_pred[train_anomalies]
test_confidence = y_test_pred[test_anomalies]

# Get latitude and longitude for anomalies
train_anomalies_lat_lon = lat_train[train_anomalies]
test_anomalies_lat_lon = lat_test[test_anomalies]

# Debugging: Print lengths to ensure they match
print(f"Train anomalies latitude/longitude shape: {train_anomalies_lat_lon.shape}")
print(f"Train confidence shape: {train_confidence.shape}")
print(f"Test anomalies latitude/longitude shape: {test_anomalies_lat_lon.shape}")
print(f"Test confidence shape: {test_confidence.shape}")

# Print anomalies and their confidence scores
print("Train anomalies:")
for i, (lat_lon, conf) in enumerate(zip(train_anomalies_lat_lon, train_confidence)):
    lat, lon = lat_lon
    print(f"Anomaly {i + 1}: Latitude={lat}, Longitude={lon}, Confidence={conf:.4f}")

print("\nTest anomalies:")
for i, (lat_lon, conf) in enumerate(zip(test_anomalies_lat_lon, test_confidence)):
    lat, lon = lat_lon
    print(f"Anomaly {i + 1}: Latitude={lat}, Longitude={lon}, Confidence={conf:.4f}")

# Measure execution time
print(f"\nExecution time: {time.time() - start:.2f} seconds")


Train anomalies latitude/longitude shape: (8, 2)
Train confidence shape: (8,)
Test anomalies latitude/longitude shape: (0, 2)
Test confidence shape: (0,)
Train anomalies:
Anomaly 1: Latitude=8.490000000000002, Longitude=75.67, Confidence=0.9984
Anomaly 2: Latitude=29.24, Longitude=90.92, Confidence=0.9995
Anomaly 3: Latitude=35.24, Longitude=93.42, Confidence=0.9977
Anomaly 4: Latitude=29.74, Longitude=88.67, Confidence=0.9975
Anomaly 5: Latitude=7.490000000000002, Longitude=70.17, Confidence=0.9971
Anomaly 6: Latitude=16.240000000000002, Longitude=69.42, Confidence=1.0000
Anomaly 7: Latitude=7.240000000000002, Longitude=85.92, Confidence=0.9987
Anomaly 8: Latitude=28.74, Longitude=92.17, Confidence=0.9974

Test anomalies:

Execution time: 6.90 seconds
